# *An Example With ScaPy*

ScaPy is a library that can be used to create and manipulate network packets, but it is also capable of doing minor attacks as well. There are going to be a few examples of what ScaPy can do.

- Creating basic packets
- tracerouting 
- attacks   



In [5]:
from scapy.all import *

# Define target IP and port (localhost and HTTP port)
target_ip = "127.0.0.1"
target_port = 80

# Craft the HTTP POST request with "Hello, World!" as the body
http_request = (
    b"POST / HTTP/1.1\r\n"
    b"Host: localhost\r\n"
    b"Content-Type: text/plain\r\n"
    b"Content-Length: 13\r\n\r\n"  # The length of "Hello, World!" is 13
    b"Hello, World!"
)

# Build the IP layer and TCP layer
ip_layer = IP(dst=target_ip)
tcp_layer = TCP(dport=target_port, sport=RandShort(), flags="S")

# Send SYN to initiate a TCP connection and receive SYN-ACK
syn_ack = sr1(ip_layer/tcp_layer, timeout=1)

# Complete the TCP handshake with ACK if SYN-ACK received
if syn_ack and syn_ack.haslayer(TCP) and syn_ack[TCP].flags == 0x12:
    # Craft ACK packet
    ack_packet = ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack, ack=syn_ack.seq + 1, flags="A")
    send(ack_packet)

    # Send HTTP POST request with "Hello, World!" in the body
    send(ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack, ack=syn_ack.seq + 1, flags="PA") / http_request)
    
    # Receive the server's response
    response = sr1(ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack + len(http_request), ack=syn_ack.seq + 1, flags="A"), timeout=1)
    
    if response:
        response.show()
else:
    print("Failed to establish a TCP connection.")


Begin emission
.*.
Finished sending 1 packets

Received 3 packets, got 1 answers, remaining 0 packets
.
Sent 1 packets.
.
Sent 1 packets.
Begin emission
.*
Finished sending 1 packets

Received 2 packets, got 1 answers, remaining 0 packets
###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 40
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x3cce
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ TCP ]###
     sport     = http
     dport     = 62268
     seq       = 2485215397
     ack       = 0
     dataofs   = 5
     reserved  = 0
     flags     = R
     window    = 0
     chksum    = 0xc98a
     urgptr    = 0
     options   = []



Before executing the above code wireshark was opened and set to capture mode. This experiment to prevent clutter of wireshark interface only uses the loopback interface which is for local host. Below you can see the screenshot from the wireshark network trace from above if you look closely at the screenshot you can see a ***hello, world*** in the network trace 


![Alt Text](screenshots/Screenshot%20from%202024-11-05%2021-34-43.png)
